## Setup

In [ ]:
# Import dependencies
import openai
import json
import pandas as pd
import os
import requests

# Retrieve API keys
# These should be set as environmental variables
OpenAI_API_KEY = os.getenv('OPENAI_API_KEY')
Azure_API_KEY = os.getenv('AZURE_API_KEY')

## Interacting directly with OpenAI's servers

##### Via Python package

In [ ]:
# Set API key for package
openai.api_key = OpenAI_API_KEY

In [ ]:
# Extract models available to DF, there's 64
models = openai.Model.list()
models = pd.DataFrame(models['data']).sort_values(by='id')

In [ ]:
# Moderation assessment model

moderation_resp = openai.Moderation.create(model='text-moderation-latest', input='People from whoville are fools')
# moderation_resp = openai.Moderation.create(input='Wilder sent crashing to the canvas')
# moderation_resp = openai.Moderation.create(input='Knife crime')

# This is where the model results are stored
moderation_resp

In [ ]:
# Completion model

completion_resp = openai.Completion.create(
    model='text-davinci-003',
    prompt='Complete the following phrase: Veni, Vidi,',
    temperature=0.7)

# This is where the model results are stored
completion_resp['choices'][0]['text']

##### Now execute the same models but interacting directly with the API

In [ ]:
# Define endpoint
url = 'https://api.openai.com/v1/moderations'

# Set headers including API key
headers = {
  'Authorization': f'Bearer {OpenAI_API_KEY}',
  'Content-Type': 'application/json'
}

# Set model inputs
payload = {
  'input': 'People from whoville are fools',
  'model': 'text-moderation-stable'
}

# Execute request
response = requests.post(url, headers=headers, json=payload)
response = json.loads(response.text)

response

In [ ]:
# Define endpoint
url = 'https://api.openai.com/v1/completions'

# Set headers including API key
headers = {
    'Authorization': f'Bearer {OpenAI_API_KEY}',
    'Content-Type': 'application/json'
}

# Set model inputs
payload = {
    'prompt': 'Complete the following phrase: Veni, Vidi, ',
    'model': 'text-davinci-003',
    'temperature': 0.7
}

# Execute request
response = requests.post(url, headers=headers, json=payload)
response = json.loads(response.text)

# This is where the model results are stored
response['choices'][0]['text']

## Interacting with an Azure deployment of OpenAI's models

##### Via Python package

In [ ]:
# Set API key for package
openai.api_key = Azure_API_KEY

# Define Azure deployment variables
azure_account_url = 'https://ascentopenai.openai.azure.com/'
deployment_name = 'test-deployment'

In [ ]:
# These fields direct the package to the Azure deployment, rather than the default OpenAI endpoint
openai.api_type = 'azure'
openai.api_base = azure_account_url
openai.api_version = '2022-12-01'

completion_resp = openai.Completion.create(
  engine=deployment_name,
  prompt='Complete the following phrase: Veni, Vidi,',
  temperature=0.5)

# This is where the model results are stored
completion_resp['choices'][0]['text']

##### Now execute the same model but interacting directly with the API

In [ ]:
# Define endpoint
url = f'{azure_account_url}openai/deployments/{deployment_name}/completions?api-version=2022-12-01'

# Set headers including API key
headers = {
    'api-key': Azure_API_KEY,
    'Content-Type': 'application/json'
}

# Set model inputs
payload = {
  'prompt': 'Complete the following phrase: Veni, Vidi,',
  'temperature': 0.5
}

# Execute request
response = requests.post(url, headers=headers, json=payload)
response = json.loads(response.text)

# This is where the model results are stored
response['choices'][0]['text']